We will be using simpletransformers for training and testing our question and answer model.
The transformers library was downloaded by the following command:
%pip install simpletransformers --> needed for installing simple transformers on jupyter

The above needs pre-installation of pytorch to be fully accessible.

In [156]:
#different frequently required libraries for data preprocessing
import pandas as pd
import numpy as np

#for checking the metric for "ANSWER_POSSIBLE"
from sklearn.metrics import accuracy_score

Importing the file. It is to be noted that only 60 percent of the entire data-set was used for faster model building and coherent checking.

In [12]:
train_file=pd.read_csv("/Users/akarshankapoor/Desktop/Development/ML:AI/train_data.csv")
train_file=train_file.drop(train_file.columns[0],axis=1)

train_file=train_file[0:int(6*len(train_file)/10)]

In [13]:
pd.DataFrame(train_file)

,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526]
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505]
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360]
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276]
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290]
...,...,...,...,...,...,...
45028,Galicia_(Spain),The establishment of the Santa Hermandad in 14...,What percentage of Castille's total earnings w...,True,['10%'],[407]
45029,Galicia_(Spain),The establishment of the Santa Hermandad in 14...,"In spite of these wars, which exports did Gali...",True,"['sardines, wood, and some cattle and wine']",[753]
45030,Galicia_(Spain),After the rupture of the wars with Portugal an...,War broke out with which other countries?,True,['Portugal and Catalonia'],[35]
45031,Galicia_(Spain),After the rupture of the wars with Portugal an...,When did the Galician Junta more often stand u...,True,['second half of the 17th century'],[369]


As for any model that is created... it was divided into 2 parts... the train and the test part. 
The QuestionAnsweringModel within simple transoformers has aspects for training, testing and predicting. The prediction file was provided later. 70 percent of the extracted file was used for training.

In [14]:
file_len=len(train_file)
train_test_ratio=0.7
value_id=int(train_test_ratio*file_len)

In [15]:
train=pd.DataFrame(train_file[:value_id])
test=pd.DataFrame(train_file[value_id:]).reset_index(drop=True)

The data provided did not have the format specified for training the model, hence it was converted into a list of dictionaries according to the format specified.....
This had to be done both with the train and test data......

In [16]:
#Finding unique Paragraphs across the dataset:
train_inst=train["Paragraph"].unique()

training_data=[]
id=1
#According to each paragraph context the questions were assigned unique IDs and were tokenized into "is_impossible" and "answer_text" format as required.
for inst in train_inst:
    dictionary={}
    current=train.loc[train['Paragraph'] == inst]
    current=current.reset_index(drop=True)
    dictionary["context"]=inst
    dictionary["qas"]=[]
    for values in range(len(current)):
        qas_dict={}
        qas_dict["id"]=str(id)
        qas_dict["is_impossible"]=~current["Answer_possible"][values]
        qas_dict["question"]=current["Question"][values]
        qas_dict["answers"]=[]
        if(qas_dict["is_impossible"]==False):
            answer_dict={}
            x=len(current["Answer_text"][values])
            y=len(current["Answer_start"][values])
            answer_dict["text"]=current["Answer_text"][values][2:x-2]
            answer_dict["answer_start"]=int(current["Answer_start"][values][1:y-1])
            qas_dict["answers"].append(answer_dict)
        id+=1
        dictionary["qas"].append(qas_dict) 
    training_data.append(dictionary)

In [167]:
#The same process as above was repeated again for the test data set.
test_inst=test["Paragraph"].unique()
testing_data=[]
id=1
for inst in test_inst:
    dictionary={}
    current=test.loc[test['Paragraph'] == inst]
    current=current.reset_index(drop=True)
    dictionary["context"]=inst
    dictionary["qas"]=[]
    for values in range(len(current)):
        qas_dict={}
        qas_dict["id"]=str(id)
        qas_dict["is_impossible"]=~current["Answer_possible"][values]
        qas_dict["question"]=current["Question"][values]
        qas_dict["answers"]=[]
        if(qas_dict["is_impossible"]==False):
            answer_dict={}
            x=len(current["Answer_text"][values])
            y=len(current["Answer_start"][values])
            answer_dict["text"]=current["Answer_text"][values][2:x-2]
            answer_dict["answer_start"]=int(current["Answer_start"][values][1:y-1])
            qas_dict["answers"].append(answer_dict)
        id+=1
        dictionary["qas"].append(qas_dict) 
    testing_data.append(dictionary)


In [168]:
#The final length of test and train data set:
print(len(testing_data))
print(len(training_data))

2586
6967


Importing the question answering model and the logging statistics for evaluating the train part of the model.

In [6]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [19]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [26]:
# Configuring the model... the model parameters were specified as follows:
model_devrev= QuestionAnsweringArgs()
model_devrev.train_batch_size= 8
model_devrev.evaluate_during_training= False
model_devrev.num_train_epochs=1
model_devrev.reprocess_input_data= True
model_devrev.n_best_size=5
model_devrev.learning_rate=3e-5

finmod= QuestionAnsweringModel("distilbert","distilbert-base-cased",args=model_devrev,use_cuda=False)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on

In [27]:
#The model was then trained on the training data_set after configuring into the required format
finmod.train_model(training_data)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
Could not find answer: '"This Can't Be Life",' vs. '"This Can\'t Be Life"'
Could not find answer: '"Hey Mama", as well as a cover of Journey's "Don't Stop Believin'", to' vs. '"Hey Mama", as well as a cover of Journey\'s "Don\'t Stop Believin\'"'
Could not find answer: 'John Lennon's "Imagine"' vs. 'John Lennon\'s "Imagine"'
Could not find answer: '6'2"' vs. '6\'2"'
Could not find answer: '5'10"' vs. '5\'10"'
Could not find answer: '"What Obama Isn't: Black Like Me."' vs. '"What Obama Isn\'t: Black Like Me."'
Could not find answer: '"It's House"' vs. '"It\'s House"'
Could not find answer: 'Steve "Silk" Hurley's "Jack Your Body"' vs. 'Steve "Silk" Hurley\'s "Jack Your Body"'
Could not find answer: '"Baseball's Sad Lexicon,"' vs. '"Baseball\'s Sad Lexicon,'
Could not find answer: '"which attempts to capture information such as 'what', 'when' and 'where'". With' vs. '"which attempts to cap

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/3971 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Training of distilbert model complete. Saved to outputs/.


(3971, 1.6060433517067356)

In [23]:
#The model was locally saved into the outputs directory and and was loaded for the testing part.
finmod=QuestionAnsweringModel("distilbert","outputs/",use_cuda=False)

#The remaining 30 percent of the data was tested upon to check if the model had been trained properly.
result,text=finmod.eval_model(testing_data)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 13510/13510 [00:00<00:00, 1166763.72it/s]


Running Evaluation:   0%|          | 0/1712 [00:00<?, ?it/s]

In [31]:
#The results are obtained below.....the model performed reasonably well with very less incorrect reports
print(result)

{'correct': 5758, 'similar': 6946, 'incorrect': 806, 'eval_loss': -6.927762973531384}


The prediction file was provided later. To make the the model ready for prediction on the provided file...a sample was tested out .... to confirm the accuracy of the model and check the required format.

In [141]:
to_predict = [
    {
        "context": "Peter Parker is Spiderman",
        "qas": [
            {
                "question": "Who is Spiderman?",
                "id": "0",
            }
        ],
    }
]


In [142]:
answers, probabilities = finmod.predict(to_predict)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 19328.59it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [144]:
#The answer for the above question can be seen below.
print(answers)
print(probabilities)

[{'id': '0', 'answer': ['Peter Parker', '', 'Parker', 'Peter']}]
[{'id': '0', 'probability': [0.960883934404576, 0.032658683934185036, 0.003435587910957817, 0.0017487183959488306]}]


The final test file was loaded into model. This file again had to be made into the required format for prediction.
This was done again by creating a list of dictionaries regarding the "Paragraphs" and their "Questions"

In [93]:
final_testfile=pd.read_csv("/Users/akarshankapoor/Desktop/Development/ML:AI/test_data/answers.csv")

In [99]:
test_inst=final_testfile["context"].unique()
finaltesting_data=[]
id=1
for inst in test_inst:
    dictionary={}
    current=final_testfile.loc[final_testfile['context'] == inst]
    current=current.reset_index(drop=True)
    dictionary["context"]=inst
    dictionary["qas"]=[]
    for values in range(len(current)):
        qas_dict={}
        qas_dict["id"]=str(id)
        qas_dict["question"]=current["question"][values]
        id+=1
        dictionary["qas"].append(qas_dict) 
    finaltesting_data.append(dictionary)

In [100]:
answer,prob=finmod.predict(finaltesting_data)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|██████████| 66/66 [00:00<00:00, 478107.19it/s]


Running Prediction:   0%|          | 0/9 [00:00<?, ?it/s]

The answers returned by finmod were in a different format than required for checking the accuracy so they were converted in to the required format so they were converted into the required dataframe format.

In [160]:
result=[]
result_dict={}
for vals in range(len(answer)):
    if(answer[vals]["answer"][0]!=""):
       result_dict["answer_possible"]=True
       result_dict["answer"]=answer[vals]["answer"][0]
    else:
        result_dict["answer_possible"]=False
        result_dict["answer"]="[]"
    result.append(result_dict)
    result_dict={}

    

In [162]:
#The result of the answers Dataframe achieved from the finmod model.
result=pd.DataFrame(result)
result

,answer_possible,answer
0,True,Felicitas Hayek (née von Juraschek). Friedrich
1,True,vary but most hatch from eggs
2,True,Tucson Kitchen Musicians Association
3,False,[]
4,True,first Saturday and Sunday of May
...,...,...
61,True,Akira Toriyama
62,True,Toei Animation
63,False,[]
64,True,Gohan


The accuracy score is printed below: 
The accuracy achieved was significantly higher than expectation due to the no. of epoch being less and also use of lesser amount of training data.

In [163]:
print(accuracy_score(final_testfile["answer_possible"],result["answer_possible"]))

0.6212121212121212


The more challenging part of the probelem statement required that that the question had to be answered from a given set of paragraphs, i.e we had to check whether the question could be answered from any of the given set of paragraphs. 

A naive way to solve the above will be to simply predict for each question given a particular context. This method has been implemented below....

In [39]:
paragraphs=pd.read_csv("/Users/akarshankapoor/Desktop/Development/ML:AI/test_data/paragraphs.csv")

In [40]:
questions=pd.read_csv("/Users/akarshankapoor/Desktop/Development/ML:AI/test_data/questions.csv")

In [ ]:
prediction=[]
for question in questions["question"]:
    final_question=""
    final_context=""
    final_answer=""
    is_possible=""
    for context in paragraphs["context"]:
        to_predict[0]["context"]=context
        to_predict[0]["qas"][0]["question"]=question
        answers, probabilities = finmod.predict(to_predict)
        if(answers[0]["answer"][0]!="" and probabilities[0]["probability"][0] > 0.5):
            final_question=question
            final_context=context
            final_answer=answers[0]["answer"][0]
            is_possible="True"
            break
        else:
            final_question=question
            final_context="[]"
            final_answer="[]"
            is_possible="False"
    prediction.append([final_question,final_context,final_answer,is_possible])
    print("Question done")
    print(final_question)
    print(final_answer)
    print(final_context)
    print(is_possible)

The above method had to be stopped early due to less available time.

However it can be improvised upon by using a "Keyword Tokenizer" inside the same simple transformers library. This method will identify the key term on which the paragraph is based upon and will then iterate through each of the provided questions thus reducing the amount of computation time.

The above could be implemented in some further additional time. 